In [42]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
import time
import random


In [52]:
def request_articles(datetime = datetime.today(), url = "https://www.faz.net/suche/?query=&ct=article&ct=blog&ct=storytelling&author=&from=&to="):
    url = re.sub("from=.*&", "from=" + (datetime - timedelta(days=1)).strftime("%d.%m.%Y") + "&", url)
    url = re.sub("to=.*", "to=" + datetime.strftime("%d.%m.%Y"), url)
    print("Requesting: ", url)

    page = urlopen(url)
    soup = BeautifulSoup(page, "html.parser")

    links = soup.find_all("a")

    try:
        page = int(re.search("/s(\d).html", url).groups()[0])
    except:
        page = 1

    links_articles = [link.get("href") for link in links if link.get("href") and re.match("https://.*/suche/.{4,}html", link.get("href"))]

    print("Articles count: ", len(links_articles))
    for arcticle in links_articles:
        print(arcticle)

    
    regex = "https://.*/suche/s" + str(page + 1) + ".html.*"

    print ("pages regex: ", regex)

    links_pages = [link for link in links if link.get("href") and re.match(regex, link.get("href"))]
    print("Pages links: ", len(links_pages))
    for pages in links_pages:
        print(pages.get("href"))

    if len(links_pages) > 0:
        print("Found a next page. Requesting: ", links_pages[0].get("href"))
        time.sleep(random.uniform(2, 5))
        links_articles.extend(request_articles(url=links_pages[0].get("href")))

    return links_articles

In [53]:
articles = request_articles()

Requesting:  https://www.faz.net/suche/?query=&ct=article&ct=blog&ct=storytelling&author=&from=11.11.2023&to=12.11.2023
Articles count:  20
https://www.faz.net/suche/die-gewinnerin-neige-sinno-gewinnt-den-prix-femina-19304507.html
https://www.faz.net/suche/ukraine-liveticker-israel-will-luftverteidigungssystem-an-finnland-verkaufen-faz-19030454.html
https://www.faz.net/suche/nfl-in-frankfurt-wie-die-eintracht-die-football-show-erkaempfte-19304919.html
https://www.faz.net/suche/fendt-verkauft-mehr-traktoren-und-bietet-nun-ein-e-modell-an-19308566.html
https://www.faz.net/suche/unfall-nach-darmstadt-heimspiel-toedlicher-verkehrsunfall-faz-skyline-blick-19267109.html
https://www.faz.net/suche/einsamkeit-ist-auch-ein-politisches-problem-19305870.html
https://www.faz.net/suche/herzblatt-geschichten-basler-will-oezil-die-einreise-verweigern-19304975.html
https://www.faz.net/suche/familientipp-mit-alpakas-im-spessart-wandern-19300592.html
https://www.faz.net/suche/liveticker-zum-krieg-in-naho

In [57]:
len(articles)

142